In [1]:
import numpy as np
import scipy
import sympy as sp
import torch
from tqdm import tqdm

import plotly.graph_objects as go
from qiskit.opflow import I, X, Y, Z, H

from qaoa import *

import pickle
import time
import warnings
warnings.filterwarnings("ignore")

from rich import print
from rich import pretty

pretty.install

/var/folders/1z/z41sxch92wz_jmlsyy10q6680000gn/T/ipykernel_9119/89265748.py:8: DeprecationWarning: The ``qiskit.opflow`` module is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/opflow_migration.
  from qiskit.opflow import I, X, Y, Z, H


<function rich.pretty.install(console: Optional[ForwardRef('Console')] = None, overflow: 'OverflowMethod' = 'ignore', crop: bool = False, indent_guides: bool = False, max_length: Optional[int] = None, max_string: Optional[int] = None, max_depth: Optional[int] = None, expand_all: bool = False) -> None>

In [2]:
def graph_energy(energy, points = None):
    res = len(energy)
    s = np.linspace(0, 2 * np.pi, res + 1)[:-1]
    
    fig = go.Figure(data=[
            go.Surface(x=s, y=s, z=energy, showscale=False,
                       colorscale='Viridis')])

    
    if points is not None:
        px, py, p_energy = points
        fig.add_trace(go.Scatter3d(x=px, y=py, z=p_energy,
                                   mode='markers'))

    fig.update_scenes(
        xaxis_title='Gamma',
        yaxis_title='Beta',
        zaxis_title='Energy'
    )

    fig.update_layout(title='p = 1 Energy Landscape',
                    autosize=False,
                    width=1000, height=700,
                    )
    fig.show()

def save(filename, data):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)

def load(filename,):
    with open(filename, 'rb') as f:
        return pickle.load(f)

In [3]:
"""
Schwinger
"""

select = {'N': 2, #qubits
          'g' : 4,  #coupling
          'm' : 2,  #bare mass
          'a' : 0.25, #lattice spacing
          'theta' : 0, #topological term
          'mixer_type' : 'Y', # type of mixer {'X', 'Y', 'XY'}
         }

model = QuantumModel('schwinger', 'Standard')


ham = model.make(select)
schwinger_target = ham.target
ham

driver:
1.0 * II
+ 2.0 * IZ
- 1.0 * ZI
mixer:
1.0 * IY
+ 1.0 * YI
target:
1.0 * II
+ 2.0 * IZ
- 1.0 * ZI
+ 1.0 * XX
+ 1.0 * YY

In [49]:
numqaoa = QAOA_Numpy(ham)

In [13]:
res = 20
energy_scape = numqaoa.energy_landscape(res, 2)

100%|██████████| 160000/160000 [02:24<00:00, 1104.03it/s]


In [24]:
graph_energy(energy_scape, points = None)

In [18]:
np.count_nonzero(np.isclose(energy_scape, np.min(energy_scape)))

energy_minimas = np.array(np.where(np.isclose(energy_scape, np.min(energy_scape)))).T * 2 * np.pi / 32

In [14]:
def landscape_minimas(landscape, filter = True):

    dims = len(landscape.shape)
    res = landscape.shape[0]

    minimas = np.full(landscape.shape, True)

    for dim in range(dims):
        less_right = landscape <= np.roll(landscape, 1 , dim)
        less_left = landscape <= np.roll(landscape, -1 , dim)

        axis_minimas = np.logical_and(less_right, less_left)
        minimas = np.logical_and(minimas, axis_minimas)

    if filter:
        global_minimas = np.isclose(landscape, np.min(landscape))
        minimas = np.logical_and(minimas, global_minimas)

    min_energy = landscape[minimas]

    points = np.array(np.where(minimas)).T * (2*np.pi) / res
    gammas, betas = np.hsplit(points, 2)

    return gammas, betas, min_energy

In [15]:
gammasp1, betasp1, min_energyp1 = landscape_minimas(energy_scape[:, 0, :, 0], filter=True)
gammasp2, betasp2, min_energyp2 = landscape_minimas(energy_scape, filter=True)

In [43]:
dx = 1e-11

def optimize(gamma, beta, eps=5e-3, tol=1e-5):
    assert len(gamma) == len(beta)
    p = len(gamma)

    gamma, beta = gamma.copy(), beta.copy()

    gamma_gradient = np.ones(p)
    beta_gradient = np.ones(p)

    count = 0

    energy_history = [numqaoa.energy(gamma, beta)]
    gamma_history = [gamma]
    beta_history = [beta]

    while count == 0 or (energy_history[-1] < energy_history[-2] and (energy_history[-2] - energy_history[-1]) > tol):
        for i in range(p):
            plus_gamma = gamma.copy()
            minus_gamma = gamma.copy()

            plus_gamma[i] += dx
            minus_gamma[i] -= dx
            
            gamma_gradient[i] = (numqaoa.energy(plus_gamma, beta) - numqaoa.energy(minus_gamma, beta)) / (2 * dx)

            plus_beta = beta.copy()
            minus_beta = beta.copy()

            plus_beta[i] += dx
            minus_beta[i] -= dx
            
            beta_gradient[i] = (numqaoa.energy(gamma, plus_beta) - numqaoa.energy(gamma, minus_beta)) / (2 * dx)

        count += 1
        gamma -= gamma_gradient * eps
        beta -= beta_gradient * eps
        
        energy_history.append(numqaoa.energy(gamma, beta))
        gamma_history.append(gamma)
        beta_history.append(beta)

        print(f"{energy_history[-1]:.6}", end="\r")

    return gamma, beta


In [48]:
new_gammasp1 = []
new_betasp1 = []

for gamma, beta in zip(gammasp1, betasp1):
    print(f"Minima {i}:")
    new_gamma, new_beta = optimize(gamma, beta)
    print("")

    new_gammasp1.append(new_gamma)
    new_betasp1.append(new_beta)

new_gammasp1 = np.array(new_gammasp1)
new_betasp1 = np.array(new_betasp1)

Minima 31:
-2.0
Minima 31:
-2.0
Minima 31:
-2.0
Minima 31:
-2.0


In [44]:
new_gammasp2 = []
new_betasp2 = []

for i, (gamma, beta) in enumerate(zip(gammasp2, betasp2)):
    print(f"Minima {i}:")
    new_gamma, new_beta = optimize(gamma, beta)
    print("")

    new_gammasp2.append(new_gamma)
    new_betasp2.append(new_beta)

new_gammasp2 = np.array(new_gammasp2)
new_betasp2 = np.array(new_betasp2)

Minima 0:
-2.08727
Minima 1:
-2.08727
Minima 2:
-2.08727
Minima 3:
-2.08727
Minima 4:
-2.08727
Minima 5:
-2.08728
Minima 6:
-2.08727
Minima 7:
-2.08727
Minima 8:
-2.08727
Minima 9:
-2.08727
Minima 10:
-2.08727
Minima 11:
-2.08727
Minima 12:
-2.08727
Minima 13:
-2.08727
Minima 14:
-2.08727
Minima 15:
-2.08727
Minima 16:
-2.08727
Minima 17:
-2.08727
Minima 18:
-2.08727
Minima 19:
-2.08727
Minima 20:
-2.08727
Minima 21:
-2.08727
Minima 22:
-2.08727
Minima 23:
-2.08727
Minima 24:
-2.08727
Minima 25:
-2.08727
Minima 26:
-2.08727
Minima 27:
-2.08727
Minima 28:
-2.08727
Minima 29:
-2.08728
Minima 30:
-2.08727
Minima 31:
-2.08727


In [244]:
new_gammasp1to2 = []
new_betasp1to2 = []

for gamma, beta in tqdm(zip(new_gammasp1, new_betasp1)):
    new_gamma, new_beta = optimize([gamma[0], 1e-1], [beta[0], 1e-1])

    new_gammasp1to2.append(new_gamma)
    new_betasp1to2.append(new_beta)

new_gammasp1to2 = np.array(new_gammasp1to2)
new_betasp1to2 = np.array(new_betasp1to2)

4it [03:39, 54.81s/it]


In [264]:

for gamma1 in new_gammasp1to2:
    penalty = np.infty
    closest = gamma1
    for gamma2 in new_gammasp2:
        if np.sum((gamma1 - gamma2)**2) < penalty:
            closest = gamma2
            penalty = np.sum((gamma1 - gamma2)**2)
    print(gamma1, closest)

[1.61280463 0.15142585] [1.50763236 0.12423114]
[1.61281701 0.15141766] [1.50763236 0.12423114]
[4.75440952 0.1514169 ] [4.6492251 0.1242311]
[4.75440967 0.15141675] [4.6492251 0.1242311]


In [267]:
1.61280463 - 1.50763236

0.10517227000000018

In [279]:
(2*np.pi) / 64

0.09817477042468103

In [263]:
np.sum((new_gammasp1to2[0] - new_gammasp2[0]) ** 2)

0.011800776693176607

In [253]:
new_gammasp2

array([[1.50763232, 0.12423099],
       [1.50763223, 0.12423141],
       [1.50763198, 0.12423174],
       [1.50763236, 0.12423114],
       [1.50763225, 3.26582377],
       [1.50763229, 3.26582426],
       [1.50763214, 3.26582386],
       [1.50763236, 3.26582393],
       [1.63396016, 3.01736156],
       [1.63396056, 3.01736111],
       [1.63396036, 3.01736122],
       [1.63396034, 3.01736185],
       [1.63396054, 6.15895361],
       [1.63396047, 6.15895334],
       [1.63396065, 6.15895339],
       [1.63396069, 6.15895346],
       [4.64922488, 0.12423141],
       [4.64922482, 0.1242313 ],
       [4.6492251 , 0.1242311 ],
       [4.64922466, 0.12423134],
       [4.64922482, 3.26582406],
       [4.64922484, 3.26582402],
       [4.64922493, 3.26582366],
       [4.64922502, 3.26582373],
       [4.77555306, 3.0173614 ],
       [4.77555272, 3.01736151],
       [4.77555312, 3.01736142],
       [4.77555306, 3.01736149],
       [4.77555326, 6.1589533 ],
       [4.77555343, 6.1589533 ],
       [4.

In [246]:

for gamma, beta in tqdm(zip(new_gammasp1to2, new_betasp1to2)):
    print(numqaoa.energy(gamma, beta))


4it [00:00, 46.38it/s]

-2.068844868090013
-2.0688525943660006
-2.0688527369102996
-2.0688527736424223


In [248]:
for gamma, beta in tqdm(zip(new_gammasp2, new_betasp2)):
    print(numqaoa.energy(gamma, beta))


6it [00:00, 51.90it/s]

1.1668395707522587
1.1668442786628863
1.1668501296691431
1.1668403598811754
1.1668415301415755
1.166845176625936
1.1668436528514692
1.166841502821333
1.0600112982607728
1.0600134336984859
1.0600128605729844


17it [00:00, 47.32it/s]

1.0600100821297591
1.0600141173940851
1.060015278220808
1.0600151501949042
1.0600148708964863
1.1668442786628839
1.1668441426618932
1.1668388903007278
1.1668464286916131
1.166845095112911


27it [00:00, 45.14it/s]

1.1668444419824433
1.1668403055421497
1.166839788411422
1.060012086681573
1.0600114611918947
1.0600120139429898
1.060011691008581
1.0600155284099095
1.060015598217494


32it [00:00, 45.24it/s]

1.0600137304509072
1.0600138293683028


In [243]:
new_gammasp1

array([[1.57079632],
       [1.57079634],
       [4.71238898],
       [4.71238898]])

In [242]:
new_gammasp2

array([[1.50763232, 0.12423099],
       [1.50763223, 0.12423141],
       [1.50763198, 0.12423174],
       [1.50763236, 0.12423114],
       [1.50763225, 3.26582377],
       [1.50763229, 3.26582426],
       [1.50763214, 3.26582386],
       [1.50763236, 3.26582393],
       [1.63396016, 3.01736156],
       [1.63396056, 3.01736111],
       [1.63396036, 3.01736122],
       [1.63396034, 3.01736185],
       [1.63396054, 6.15895361],
       [1.63396047, 6.15895334],
       [1.63396065, 6.15895339],
       [1.63396069, 6.15895346],
       [4.64922488, 0.12423141],
       [4.64922482, 0.1242313 ],
       [4.6492251 , 0.1242311 ],
       [4.64922466, 0.12423134],
       [4.64922482, 3.26582406],
       [4.64922484, 3.26582402],
       [4.64922493, 3.26582366],
       [4.64922502, 3.26582373],
       [4.77555306, 3.0173614 ],
       [4.77555272, 3.01736151],
       [4.77555312, 3.01736142],
       [4.77555306, 3.01736149],
       [4.77555326, 6.1589533 ],
       [4.77555343, 6.1589533 ],
       [4.

In [241]:
new_gammasp1

array([[1.57079632],
       [1.57079634],
       [4.71238898],
       [4.71238898]])

In [168]:
class ParametersNode:
    def __init__(self, 
                 parameters,
                 energy):
        
        self.parameters = parameters
        self.energy = energy

        self.id = tuple(np.round(parameters, 12))

        self.p = len(parameters) // 2

        self.gammas = parameters[::2]
        self.betas = parameters[1::2]

        self.layer = None

        self.prev = {}
        self.next = {}

class NodeLayer:
    def __init__(self):
        self.nodes = {}
        self.num_nodes = 0

        self.prev = None
        self.next = None

    def process_new_nodes(self, all_new_nodes):
        partition_size = len(all_new_nodes) // len(self.nodes)
        for i, node in enumerate(self.nodes.values()):
            new_nodes = all_new_nodes[partition_size * i: partition_size * (i + 1)]

            if self.next is None:
                self.next = NodeLayer()
                self.next.prev = self

            for new_node in new_nodes:
                if new_node.id in self.next.nodes.keys():
                    new_node = self.next.nodes[new_node.id]
                else:
                    self.next.add_node(new_node)

                node.next[new_node.id] = new_node
                new_node.prev[node.id] = node

    def add_node(self, new_node: ParametersNode):
        self.nodes[new_node.id] = new_node
        self.num_nodes = len(self.nodes)

In [106]:
def closeness_to_ground(val):
    return 1 - (ham.eigen_values()[-1] - val) / (ham.eigen_values()[-1] - ham.eigen_values()[0])

closeness_to_ground(np.array(converged_energies)) * 100

array([1.87232156e-03, 1.63165820e-04, 1.90947989e-03, 7.17681748e-03,
       1.82311013e-02, 3.96170086e-01, 5.65075337e-01, 7.40732470e-01,
       1.20802452e+00])

In [131]:
import plotly.graph_objects as go

def plot_parameters(converged_parameters):
    fig = go.Figure()

    num_lines = len(converged_parameters)

    for i, parameters in enumerate(converged_parameters[::-1]):
        p = len(parameters) // 2

        gammas = parameters[::2]
        betas = parameters[1::2]

        print(p)
        
        fig.add_trace(go.Scatter(x=np.linspace(0, 1, p), y = gammas,
                                 opacity= ((i + 1) / num_lines) ** 5,
                                 mode="lines",
                                 line=dict(color='red')))
        fig.add_trace(go.Scatter(x=np.linspace(0, 1, p), y = betas,
                                 opacity= ((i + 1) / num_lines) ** 5,
                                 mode="lines",
                                 line=dict(color='blue')))
    fig.update_layout(showlegend=False, 
                    width=1200,
                    height=700,
                    title = "Parameter Curves",
                    xaxis_title = "Step (i-1)/(p-1)",
                    yaxis_title = "Gamma(red), Beta(blue)")
            
    fig.show()
    
plot_parameters(converged_parameters)

1
2
3
4
5
6
7
8
9


In [138]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=list(range(1, 1 +len(converged_energies))), y = (1 - closeness_to_ground(np.array(converged_energies[1::][::-1]))) * 100,
                                 line=dict(color='violet')))

fig.update_layout(showlegend=False,
                  width=1200,
                    height=700,
                  title = "Performance Scaling of p",
                  xaxis_title = "p - number of rounds",
                  yaxis_title = "Closeness to Ground (%)")

fig.show()

In [139]:
(1 - closeness_to_ground(np.array(converged_energies[1::][::-1]))) * 100

array([98.79197548, 99.25926753, 99.43492466, 99.60382991, 99.9817689 ,
       99.99282318, 99.99809052, 99.99983683])

In [137]:
closeness_to_ground(np.array(converged_energies[1::][::-1]))[-1]

1.6316581957775966e-06

In [68]:
def closeness_to_ground(val):
    return (ham.eigen_values()[-1] - val) / (ham.eigen_values()[-1] - ham.eigen_values()[0])

closeness_to_ground(ham.eigen_values()[0] + 1e-4) * 100

99.99629516126933

In [21]:
with open('data', 'wb') as f:
    pickle.dump(layers,f)

with open('data', 'rb') as f:
    y = pickle.load(f)

In [15]:
import torch

torch.view_as_complex()

TypeError: view_as_complex() missing 1 required positional arguments: "input"

In [96]:
size = 4
plus = torch.ones(size, 1, dtype=torch.complex128)
target_tensor = torch.matrix_exp(torch.from_numpy(ham.target.to_matrix()) * -1j)

# target_tensor = torch.ones(size,size, dtype=torch.complex128)
plus.adjoint() @ target_tensor @ plus - torch.sum(target_tensor)

tensor([[-1.1102e-16+0.j]], dtype=torch.complex128)

In [69]:
import torch

class QAOA_torch:
    def __init__(self, hamiltonian, device="cpu"):
        self.hamiltonian = hamiltonian

        self.n = hamiltonian.num_qubits
        self.N = 2 ** self.n

        self.driver = torch.from_numpy(self.hamiltonian.driver.to_matrix()).to(device)
        self.mixer = torch.from_numpy(self.hamiltonian.mixer.to_matrix()).to(device)
        self.target = torch.from_numpy(self.hamiltonian.target.to_matrix()).to(device)
        self.driver_and_mixer = torch.stack((self.driver, self.mixer))

        # self.grnd_energy = hamiltonian.eigen_values(k=1)
        # self.grnd_state = hamiltonian.eigen_vectors(k=1)
    
    def energy(self, gammas, betas):
        
        assert gammas.shape == betas.shape
        param_shape = gammas.shape

        if len(param_shape) == 1:
            batches = 1
            p = param_shape[0]
        elif len(gammas.shape) == 2:
            batches, p = param_shape
        else:
            raise RuntimeError("gammas and betas must be 1 or 2 dimensional")
        
        # (batches, N)
        psi = torch.ones(batches, self.N, dtype=torch.complex128) / np.sqrt(self.N)

        # (batches, p, 2)
        parameters = torch.stack((gammas, betas)).permute(1, 2, 0)

        # (batches, p, 2, N, N)
        hamiltonians = self.driver_and_mixer.unsqueeze(0).unsqueeze(0).repeat(batches, p, 1, 1, 1)

        hamiltonians = torch.einsum('bpc,bpcij->bpcij', parameters, hamiltonians)
        # (batches * p * 2, N, N)
        hamiltonians = hamiltonians.reshape(batches * p * 2, self.N, self.N)

        hamiltonians = torch.matrix_exp(hamiltonians * -1j)
        # (batches, p, 2, N, N)
        hamiltonians = hamiltonians.view(batches, p, 2, self.N, self.N)
        # (p, 2, batches, N, N)
        hamiltonians = hamiltonians.permute(1, 2, 0, 3, 4)

        for i in range(p):
            batch_driver, batch_mixer = hamiltonians[i]
            
            psi = torch.einsum('bij,bj->bi', batch_driver, psi)
            psi = torch.einsum('bij,bj->bi', batch_mixer, psi)

        result = torch.einsum('ij,bj->bi', self.target, psi)
        result = torch.einsum('bi,bi->b', psi.conj(), result)

        return result.real

    def energy_landscape(self, res, p):
        s = torch.linspace(0, 2 * np.pi, res + 1)[:-1]
        dims = torch.meshgrid(*[s] * (2 * p))
        coords = torch.stack([dim.flatten() for dim in dims]).T
        gammas, betas = torch.hsplit(coords, 2)

        num_chunks = (res ** (2 * p)) // 3_000_000 + 1

        gamma_chunks = torch.chunk(gammas, num_chunks)
        beta_chunks = torch.chunk(betas, num_chunks)

        energies = []
        for gamma_chunk, beta_chunk in zip(gamma_chunks, beta_chunks):
            energies.append(torch_qaoa.energy(gamma_chunk, beta_chunk))

        energies = torch.cat(energies)
        energies = energies.reshape([res] * (2 * p))
        return energies


In [70]:
torch_qaoa = QAOA_torch(ham)

In [74]:
energy_scape = torch_qaoa.energy_landscape(res=1000, p=1)

In [61]:
res = 8
p = 2

s = torch.linspace(0, 2 * np.pi, res + 1)[:-1]
dims = torch.meshgrid(*[s] * (2 * p))
coords = torch.stack([dim.flatten() for dim in dims]).T
gammas, betas = torch.hsplit(coords, 2)

num_chunks = gammas.shape[0] // 3_000_000 + 1

gamma_chunks = torch.chunk(gammas, num_chunks)
beta_chunks = torch.chunk(betas, num_chunks)

energies = []
count = 0
for gamma_chunk, beta_chunk in zip(gamma_chunks, beta_chunks):
    energies.append(torch_qaoa.energy(gamma_chunk, beta_chunk))

energies = torch.cat(energies)
energies = energies.reshape([res] * (2 * p))

In [65]:
gammas.shape[0]

4096

In [66]:
res ** (2 * p)

4096

In [62]:
energy_scape = numqaoa.energy_landscape(res, p)

100%|██████████| 4096/4096 [00:05<00:00, 803.42it/s] 


In [63]:
np.sum((energy_scape - energies.numpy()) ** 2)

2.9022024696533e-09